In [1]:
# Este código foi desenvolvido usando o Google Collaboratory
# e roda utilizando arquivos em um Google Drive

########################################################
##### ANTES DE RODAR DEFINA AS SEGUINTES VARIÁVEIS #####
########################################################

# Endereço (no drive) do arquivo csv da lookup table gerado no LibRadtran:
# O arquivo deve conter PWV, Cosseno do ângulo zenital, Airmass e Radiância (separados por vírgulas).
data_table = '/content/drive/My Drive/Doutorado/Dados/Tabelas/lt_2.0_10082018_CH4.csv'

# Valores inicial, final e incremento de PWV usado na lookup table:
Vini = 4.0
Vfin = 25.0
Vinc = 0.1

# Valores inicial, final e incremento do cosseno do angulo zenital usados nas simulações:
Cini = -1.00
Cfin = -0.50
Cinc = 0.025

# Dados do ASIVA - Endereço da pasta onde estão os arquivos de diferenças de contagens no drive (filtrando dados do canal 4):
path4 = '/content/drive/My Drive/Doutorado/Dados/10-08-2018_Noturno/*diff_4.fits'

# Código de identificação do arquivo de massas de ar no drive:
am_id = '1imaUyPaSLW08b0uQlHFBRE-GKztXVjcF'

# Código de identificação do arquivo de ganhos no drive:
gn_id = '1anDF0bAK4Mk3JJmwCwvUhoVyfvFG_X_-'

In [2]:
# Imports gerais:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import glob

from scipy.integrate import simps
from scipy.interpolate import interp1d
import scipy.integrate as integrate

from scipy.optimize import curve_fit
from matplotlib.mathtext import DejaVuSerifFontConstants
import time

# Função que retorna o file ID a partir do endereço no drive
!pip install kora
from kora.xattr import get_id

# Montando o drive para acessar os arquivos csv:
from google.colab import drive

drive.mount('/content/drive')


# Autencicação para acessar arquivos FITS:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from astropy.io import fits

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 34.6 MB/s eta 0:00:00
Mounted at /content/drive


In [3]:
# Método para integração da função de Planck

# Transmitância para CH1 e CH4
lam1 = np.array([7.5,	7.6,	7.7,	7.8,	7.9,	8,	8.1,	8.2,	8.3,	8.4,	8.5,	8.6,	8.7,	8.8,	8.9,	9,	9.1,	9.2,	9.3,	9.4,	9.5,	9.6,	9.7,	9.8,	9.9,	10])
t1 = np.array([0,	0.00099387241,	0.0037543261,	0.016397096,	0.067411996,	0.16470027,	0.23448369,	0.28485084,	0.32886598,	0.36326829,	0.37955591,	0.37672785,	0.37035781,	0.38823766,	0.44920856,	0.49278599,	0.43715978,	0.35334194,	0.26878348,	0.13254657,	0.047449771,	0.014290923,	0.0063342853,	0.0017594688,	0,	0])

lam4 = np.array([9.5, 9.6, 9.7, 9.8, 9.9, 10, 10.1, 10.2, 10.3, 10.4, 10.5, 10.6, 10.7, 10.8, 10.9, 11, 11.1, 11.2, 11.3, 11.4, 11.5, 11.6, 11.7, 11.8, 11.9, 12, 12.1, 12.2, 12.3, 12.4, 12.5, 12.6, 12.7, 12.8, 12.9, 13])
t4 = np.array([0.0000000, 6.1639049e-03, 1.2173898e-02, 2.7315959e-02, 6.6134155e-02, 1.5677769e-01, 3.1759289e-01, 5.0121820e-01, 6.2649029e-01, 6.6445202e-01, 6.7786473e-01, 7.0090240e-01, 7.2765547e-01, 7.4226588e-01, 7.3759609e-01, 7.0764691e-01, 6.6859865e-01, 6.3946754e-01, 6.2751961e-01, 6.2927783e-01, 6.3590199e-01, 6.3487315e-01, 6.2753671e-01, 6.1468023e-01, 5.9755743e-01, 5.8034897e-01, 5.6401944e-01, 5.4896957e-01, 5.3071028e-01, 4.8840195e-01, 3.5360095e-01, 1.7142840e-01, 7.3776573e-02, 3.4712266e-02, 1.7007021e-02, 0.0000000])

# Integral da transmitância.
It1 = simps(t1, lam1)
It4 = simps(t4, lam4)

# Temperatura de 0 a 500 K - Passo de 0.1 K.
T = np.linspace(0, 500, num=5001)

# BB = Integral(f(T)) / It
def f1(T):
    return (t1 * 1.19e+08 * lam1**-5) / (np.exp(1.44e+04 / (lam1 * T)) - 1)

def f4(T):
    return (t4 * 1.19e+08 * lam4**-5) / (np.exp(1.44e+04 / (lam4 * T)) - 1)

BB1 = [simps(f1(x), lam1)/It1 for x in T]
BB4 = [simps(f4(x), lam4)/It4 for x in T]

# Função para fazer a interpolação
BB1_interp = interp1d(T, BB1)
BB4_interp = interp1d(T, BB4)


<ipython-input-3-66dc4ff67ff2>:19: RuntimeWarning: divide by zero encountered in true_divide
  return (t1 * 1.19e+08 * lam1**-5) / (np.exp(1.44e+04 / (lam1 * T)) - 1)
<ipython-input-3-66dc4ff67ff2>:19: RuntimeWarning: overflow encountered in exp
  return (t1 * 1.19e+08 * lam1**-5) / (np.exp(1.44e+04 / (lam1 * T)) - 1)
<ipython-input-3-66dc4ff67ff2>:22: RuntimeWarning: divide by zero encountered in true_divide
  return (t4 * 1.19e+08 * lam4**-5) / (np.exp(1.44e+04 / (lam4 * T)) - 1)
<ipython-input-3-66dc4ff67ff2>:22: RuntimeWarning: overflow encountered in exp
  return (t4 * 1.19e+08 * lam4**-5) / (np.exp(1.44e+04 / (lam4 * T)) - 1)


In [4]:
# Obtendo um array com os códigos de identificação de todos os arquivos em uma pasta:

# Obtem todos os endereços dos diff em ordem alfabética:
file_path4 = glob.glob(path4)
file_path4 = sorted(file_path4)

data_diff4 = []

for p4 in file_path4:
  data_diff4.append(get_id(p4))

print(data_diff4)

['1Of_V4TlLFCXWDMKDY2u7LJ68Ra5DpIVQ', '1QlF_E9pMW8xiCGDpME4-QKTA3b7pxNk1', '1Pic3dCzQ2AIaCz0vWnxlFqAUWlpNRmFp', '1tBExAqJho0zeUPl7HZSqhHDMpMLcbpks', '1XJPGcZBa8P03FyfO_5qLBwnsCmZ1PKRN', '1h9WyvdYO9wNcRDhOWMyrI4EawhUyEfU_', '1u95YNSjdLSyMcpRrMUj6gkXFHW7MdEDs', '1vq2_5MegnW9bbXrwmdIT7WVJ2V-I3nUn', '10GuU2ONW_F5x4PEHZJ4oJ9kxAnE2ISPy', '19XzVgQmFDj3qALWU0APH0o3W7xTZUNAa', '1IU5XgUTE1XStpr7IYDsY_oFiaIdr5cE4', '1ijtZzU_GwyBhrjRf60XiHOBTsi4DjN8Z', '13K_yNHxgQqOfjc1Hh_UuG80wq-u-dZGA', '1GSXU6PsLC6ALr2eUdM7F2ofteUChnd3F', '1w2DK6C0S4kqc-ok5drFO0LoIZMMADUku', '1AzgPYHlRL_aIlq0cnyaorDTN_Y1JZ1kX', '1AI4Q5hOBQGP85jRzyxkVop8fV6fbCE1p', '1n-uJpyL-iJiXKAfzHtj3yYRXDRGe7Y72', '1jcGh9SSPjZILmc1YzlOwiue_3dXoIH1H', '1qEViO1joFRv1UlYChPzvRE511CqG63ck', '1YAnIU7NOguu1AvpFKR7Dq2KsLnEEZXr5', '1i5qXFRp1i0RY52E-1JA0wAX9ch7enVmM', '1HmkVVfV29xWLaQDUpUVkt5ySGQ983RDA', '15D7F0uOrcEUHcJgoUlL7Ctlp9UJZ_9YJ', '1c6TwJK1ehHF9ooRSEd6eQF3RxPrtDk4M']


In [5]:
# Importando arquivo de airmass e ganhos do Google Drive a partir de seus códigos:

# Arquivo de airmass:
am = 'airmass.fit'
dl = drive.CreateFile({'id': am_id})
dl.GetContentFile(am)

airmass = fits.getdata(am)


# Arquivo de ganho novo (70C) - CH4.
gn4 = 'gn_ch4.fit'
dl = drive.CreateFile({'id': gn_id})
dl.GetContentFile(gn4)

gain4 = fits.getdata(gn4)

In [6]:
# Calcula o array de massas de ar usado nas simulações do libRadtran:

n = ((Cfin - Cini)/Cinc)+1
sim_cos = np.linspace(Cini, Cfin, round(n))
sim_am = -1/sim_cos

#print('sim_am = '+ str(sim_am) )

In [7]:
# Gerando o array de PWVs usados na lookup table:

num = ((Vfin - Vini)/Vinc)+1

PWV = np.linspace(Vini, Vfin, round(num))

#print('PWV = '+ str(PWV))

In [8]:
# Carregando arquivo de lookup table gerada no libRadtran

# Nomes das colunas do arquivo csv
cols=['pwv','cos','airm','rad4']

# Abrindo o arquivo csv com o pandas
ltable = pd.read_csv(data_table, skiprows=1, sep=",", names=cols, na_values=["-9999"])

In [9]:
########################################
##########  MÉTODO PRINCIPAL  ##########
########################################

# Obtém a coluna de vapor de água precipitável a partir da comparação de pontos 
# do envelope obtido de cada imagem do ASIVA com simulações da lookup table
# gerada usando o libRadtran.
# Canal 4 do ASIVA - 10 a 12 µm

## Arrays para guardar os resultados de coluna de Vapor, Tempo e indice de comparação Qui:
Vapor = []
Tempo = []
Qui = []

# Laço que percorre toda a lista de arquivos (data_diff4):
for i in range(len(data_diff4)):

  # Contador para acompanhar o andamento:
  print(str(i+1)+'/'+str(len(data_diff4)))
  
  # Carregando arquivo de diferença de contagens:
  d4 = 'diff_ch4.fit'
  dl = drive.CreateFile({'id': data_diff4[i]})
  dl.GetContentFile(d4)
  dif4 = fits.getdata(d4)

  # Carregando dados e informações do header do arquivo:
  hdul = fits.open(d4)
  hdr = hdul[0].header
  
  # Obtendo o Tempo em horas:
  eps = hdr['EPOCHS']
  t = time.gmtime(eps)

  # Obtendo temperaturas dos corpos negros interno e externo:
  Ti = hdr['INTBBTMP']
  Te = hdr['EXTBBTMP']

  # Diferença de contagens e ganhos na região do corpo negro externo:
  d4reg = dif4[499:504,259:274]
  g4reg = gain4[499:504,259:274]

  # Medianas calculadas na região do corpo negro externo:
  df4 = np.median(d4reg)
  g4ext = np.median(g4reg)
  
  # Calculo da radiância do corpo negro de referência (interno) na região do corpo negro externo:
  Ti_K = Ti + 273.15
  Ri4 = BB4_interp(Ti_K)

  # Valor de contagens teórico para o corpo negro de referência na região da imagem do corpo negro externo:
  c4int = Ri4*g4ext

  # Calculo da radiância do corpo negro externo:
  Te_K = Te + 273.15
  Re4 = BB4_interp(Te_K)

  # Valor de contagens teórico para o corpo negro externo:
  c4ext = Re4*g4ext

  # Offset:
  off4 = c4ext - (df4 + c4int)

  #######################
  ### RADIÂNCIA - CH4 ###
  #######################
  radiance4 = ((dif4 + off4)/gain4) + Ri4

  #############################################
  ### Retirada de nuvens e limpeza de dados ###
  #############################################
  rad_limpa4 = np.copy(radiance4)

  # Retirando dados da região externa à imagem:
  fora = np.where(airmass < 1)
  rad_limpa4[fora] = np.nan

  # Mediana da radiância em torno de 3 massas de ar:
  mediana_am3 = np.nanmedian(radiance4[np.logical_and(airmass < 3.01, airmass > 2.99)])

  # Função que calcula o desvio padrão dos pixels adjacentes ao de coordenada [i,j]:
  def desvio_pad (i, j, rad):
    corte = [rad[i-1, j-1], rad[i, j-1], rad[i+1, j-1], rad[i-1, j], rad[i, j], rad[i+1, j], rad[i-1, j+1], rad[i, j+1], rad[i+1, j+1]]
    return np.std(corte)

  # Indetifica pixels a remover e transforma em NaN
  for i in range (10, 510):
    for j in range (30, 600):
      if ((desvio_pad(i, j, radiance4) > 0.07)|(radiance4[i,j] > mediana_am3)):
        rad_limpa4[i, j] = np.nan

  #############################
  ### RADIÂNCIA LIMPA - CH4 ###
  #############################
  radiance4 = np.copy(rad_limpa4)

  # Obtenção dos valores do envelope para cada valor de airmass simulado (sim_am):
  env_L4 = []

  for x in sim_am:
    fatia = np.where((airmass > x-0.001) & (airmass <= x+0.001))
    med = np.nanmedian(radiance4[fatia])
    min = np.nanmin(radiance4[fatia])
    env_L4.append(med)

  env_L4 = np.array(env_L4)

  # Array para armazenar os valores de somatórias de diferenças quadráticas:
  Q4 = []

  for mm in PWV:
    sel4 = ((ltable.pwv > mm-0.001)&(ltable.pwv < mm+0.001))
    rad4_comp = np.array(ltable.rad4[sel4])
    q = (rad4_comp - env_L4)/(rad4_comp + env_L4)
    q2 = q*q
    Q4.append(np.sum(q2))

  Q4 = np.array(Q4)

  # Obtendo valores de PWV que minimizam as diferenças quadráticas:
  min_qui4 = np.min(Q4)
  min_index4 = np.argmin(Q4)

  # Salvando os valores obtidos nos arrays de Vapor e Tempo:
  Vapor.append(PWV[min_index4])
  Tempo.append(t[3] + (t[4]/60))



1/25
2/25
3/25
4/25
5/25
6/25
7/25
8/25
9/25
10/25
11/25
12/25
13/25
14/25
15/25
16/25
17/25
18/25
19/25
20/25
21/25
22/25
23/25
24/25
25/25


In [10]:
# Resultados:

print('PWV (mm):')
print(Vapor)
print('Horário UTC:')
print(Tempo)


PWV (mm):
[13.0, 13.0, 11.600000000000001, 9.9, 9.3, 8.0, 6.9, 6.5, 6.5, 6.5, 6.5, 6.800000000000001, 6.800000000000001, 6.9, 7.0, 6.9, 7.0, 6.9, 6.7, 6.5, 6.6, 6.4, 6.6, 6.9, 7.2]
Horário UTC:
[0.38333333333333336, 0.8833333333333333, 1.3833333333333333, 1.8833333333333333, 2.3833333333333333, 2.8833333333333333, 3.3833333333333333, 3.8833333333333333, 4.383333333333334, 4.883333333333333, 5.383333333333334, 5.883333333333333, 6.383333333333334, 6.883333333333333, 7.4, 7.9, 8.4, 8.9, 9.4, 9.9, 10.4, 10.9, 11.9, 12.4, 12.9]
